In [2]:
import torch
from calculations.calculation_cf_autograd import calculation_cf_autograd
from hamiltonians.Central import CentralBdG
from hamiltonians.Lead import Lead

In [3]:
# Lattice dimensions
Nx = 50  # Length of the system
Ny = 30  # Width of the system

# Hopping parameters
t_x = torch.tensor(1.0, dtype=torch.complex64)
t_y = torch.tensor(1.0, dtype=torch.complex64)

# Magnetic flux per plaquette (related to magnetic field)
phi = torch.tensor(0.02, dtype=torch.float32)

# Disorder potential strength
U0 = torch.tensor(0.3, dtype=torch.float32)

# Other parameters
eta = 1e-5  # Small imaginary part for regularization
max_derivative_order = 2  # Maximum order of derivatives to compute


2. **Define Simulation Parameters**

   Set up the parameters for your simulation, such as lattice dimensions, hopping parameters, magnetic field strength, and disorder potential.

   ```python

   ```

3. **Construct the Central Hamiltonian with Magnetic Field and Disorder**

   Use your `CentralBdG` class from 

Central.py

 to construct the Hamiltonian of the central region, incorporating the effects of the magnetic field and disorder.



In [4]:
# Define the magnetic vector potential function or Peierls phase
def peierls_phase(x_i, y_i, x_j, y_j, phi):
    return torch.exp(-0.5j * phi * (x_i - x_j) * (y_i + y_j))

# Create an instance of CentralBdG with appropriate parameters
central_hamiltonian = CentralBdG(
    Ny=Ny,
    Nx=Nx,
    t_y=t_y,
    t_x=t_x,
    Delta=torch.tensor(0.0, dtype=torch.complex64)
)

# Modify the Hamiltonian to include magnetic field and disorder
H_BdG = central_hamiltonian.H_full_BdG.clone()

# Apply Peierls substitution for magnetic field
# (This step requires modifying the hopping terms in H_BdG using peierls_phase)

# Add disorder potential to onsite energies
disorder = U0 * (torch.rand(Nx * Ny) - 0.5)
for idx in range(Nx * Ny):
    H_BdG[idx, idx] += disorder[idx]



4. **Define Leads Information**

   Set up the lead parameters using your `Lead` class from 

Lead.py

.



In [5]:
# Define lead parameters
lead = Lead(
    t=t_x,
    epsilon=4 * t_x,  # Adjust according to your model
    # Include other necessary parameters
)

# Leads information list
leads_info = [lead]  # Add more leads if necessary

TypeError: Lead.__init__() got an unexpected keyword argument 't'



5. **Compute Energy Values**

   Define the energy range over which you want to perform the calculations.

   ```python
   # Energy values
   E_values = torch.linspace(0.0, 0.3, steps=100)  # Adjust the range and steps as needed
   ```

6. **Calculate the Generating Function and Its Derivatives**

   Use the 

calculation_cf_autograd

 function to compute the generating function and its derivatives.



In [ ]:
# Calculate generating function and derivatives
results = calculation_cf_autograd(
    H_BdG=H_BdG,
    E_batch=E_values,
    eta=eta,
    leads_info=leads_info,
    max_derivative_order=max_derivative_order
)

# Extract results
gen_func_values_real = results['gen_func_values_real']
gen_func_values_imag = results['gen_func_values_imag']
derivatives = results['derivatives']



7. **Analyze and Visualize the Results**

   Plot the results to visualize the Quantum Hall effect and the impact of disorder.

   ```python
   import matplotlib.pyplot as plt

   # Plot the real part of the generating function
   plt.figure()
   plt.plot(E_values.cpu(), gen_func_values_real)
   plt.title('Real Part of Generating Function')
   plt.xlabel('Energy')
   plt.ylabel('Real[Log Det(Ginv_total)]')
   plt.show()

   # Plot the imaginary part of the generating function
   plt.figure()
   plt.plot(E_values.cpu(), gen_func_values_imag)
   plt.title('Imaginary Part of Generating Function')
   plt.xlabel('Energy')
   plt.ylabel('Imag[Log Det(Ginv_total)]')
   plt.show()

   # Plot derivatives if needed
   # For example, plot the first-order derivative
   first_order_derivative = derivatives['order_1']
   plt.figure()
   plt.plot(E_values.cpu(), first_order_derivative.numpy())
   plt.title('First-order Derivative of Generating Function')
   plt.xlabel('Energy')
   plt.ylabel('Derivative')
   plt.show()
   ```

8. **Interpret the Results**

   Analyze the plots to understand the formation of Landau levels, edge states, and the effect of disorder in the system.

**Note:** Ensure that all the functions and classes used are properly defined in your codebase. You might need to adjust the code to match the specific implementations of your `Hamiltonian` and `Lead` classes.